# Generating Simulated Nanoparticle Images

In this notebook, simulated nanoparticles will be generated. These will then be rotated using rotation matrices and projected onto a 2D plane. This projection will be blurred and Poisson noise will be added. The projections will be saved as images directly into Google Drive, separated into training, validation, and testing datasets.

All of my work can be found at https://github.com/javidahmed64592/Y4-Nanoparticles-Project.

# Importing the relevant libraries and configuring the figure.

In [1]:
pip install --upgrade Pillow

In [2]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

import numpy as np
import os
import cv2
import PIL
from PIL import ImageFilter
import matplotlib.pyplot as plt
import datetime

img_width, img_height = 128, 128
plt.style.use("dark_background")
fig = plt.figure(figsize=(3, 3))

<Figure size 216x216 with 0 Axes>

The images are saved directly in Google Drive.

In [3]:
from google.colab import drive
drive_path = "/content/drive"
drive.mount(drive_path, force_remount=True)

cwd = os.path.join(drive_path, "MyDrive", "Nanoparticles")
file_type = "png"

Mounted at /content/drive


# Helper functions.

The following functions are used to augment the data by randomly adjusting the brightness and channels.

In [4]:
def brightness(img, low, high):
  """
  Adjusts image brightness randomly by a factor between low and high.

  Inputs:
    img: NumPy array, image to use to adjust brightness
    low: Float, lower limit for random factor to multiply image's brightness
    high: Float, upper limit for random factor to multiply image's brightness
  
  Outputs:
    img: NumPy array, image after brightness has been adjusted
  """
  value = np.random.uniform(low, high)
  hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
  hsv = np.array(hsv, dtype = np.float64)
  hsv[:,:,1] = hsv[:,:,1] * value
  hsv[:,:,1][hsv[:,:,1] > 255]  = 255
  hsv[:,:,2] = hsv[:,:,2] * value 
  hsv[:,:,2][hsv[:,:,2] > 255]  = 255
  hsv = np.array(hsv, dtype = np.uint8)
  img = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
  return img

In [5]:
def channel_shift(img, value):
  """
  Adjusts image channels randomly by a specified value.

  Inputs:
    img: NumPy array, image to use to adjust channels
    value: Integer, adjust channels by some random integer in the range [-value, value)
  
  Outputs:
    img: NumPy array, image after channels have been adjusted
  """
  value = int(np.random.uniform(-value, value))
  img = img + value
  img[:,:,:][img[:,:,:] > 255]  = 255
  img[:,:,:][img[:,:,:] < 0]  = 0
  img = img.astype(np.uint8)
  return img

The following function is used to map a random Poisson array to be in the limits of the plot to simulate Poisson background noise.

In [6]:
def map_params(array_to_map, map_range):
  """
  Maps an array from its range to a new specified range.

  Inputs:
    array_to_map: NumPy array, array to be mapped to a new range
    map_range: NumPy array, range to use for new mapped array
  
  Outputs:
    mapped_array: NumPy array, array_to_map mapped to new map_range
  """
  y_max, y_min = np.max(array_to_map), np.min(array_to_map)
  map_max, map_min = np.max(map_range), np.min(map_range)

  mapped_array = map_min + ((array_to_map - y_min) * (map_max - map_min) / (y_max - y_min))

  return mapped_array

The following function is used to generate a list of numbers in a specified range and with a specified step between each number.

In [7]:
def generate_params(start=0, end=None, step=None, scale=1):
  """
  Adjusts image brightness randomly by a factor between low and high.

  Inputs:
    start: Integer, lower end of desired range
    end: Integer, higher end of desired range
    step: Integer, step between each number
    scale: Float or Integer, scale to multiply every number
  
  Outputs:
    List, list of numbers between start and end spaced evenly by specified step
  """
  if end is None:
    return [start * scale]

  if step is None:
    return [start * scale, end * scale]

  return [scale * (start + i * step) for i in range(1 + int((end-start)/step))]

# Creating the class.

The shape3D class generates a lattice to simulate different nanoparticle shapes. These bodies can be rotated in 3 dimensions and then projected onto a 2D plane. This plane can then have noise added to simulate Poisson noise from measuring instruments. This is then saved as an image. These functions will be shared for all lattice shapes.

In [8]:
class shape3D:
  """
  This class creates a 3D shape from a specified width and 3 angles of rotation, rx, ry, rz. The
  shape can then be projected onto 2 axes after being rotated. These projections are saved as
  images, and they can also have a Gaussian blur applied to them and those are saved in a
  subfolder.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Initialises the class.

    Inputs:
      width: Integer, width of the shape being created
      rx, ry, rz: Float, rotations in degrees about the x, y, z axes respectively
      a: Float, lattice spacing
      defect: Float between 0 and 1, percentage of defects in the shape, 0 is no defects
      pos_error: Float, error in the position of each point
    """
    self.width = width
    self.rx = rx
    self.ry = ry
    self.rz = rz
    self.defect = defect
    self.pos_error = pos_error
    self.a = a
    self.alpha = 0.25

    data = "W%s RX%s RY%s RZ%s D%s" % (self.width, self.rx, self.ry, self.rz, int(100*self.defect))
    self.name = "%s %s" % (self.name, data)

    self.generate() # Generating the shape
    self.rotate() # Rotating the shape
    self.projection2D() # Projecting the shape onto a 2D surface

  def generate(self):
    """
    Subclasses override this function to generate self.coords.
    """
    pass

  def rotate(self):
    """
    Calculates the rotation matrix to rotate the shape.
    """
    # Converting degrees to radians
    rx = np.deg2rad(self.rx)
    ry = np.deg2rad(self.ry)
    rz = np.deg2rad(self.rz)

    # Rotation matrices
    Rx = np.array([[           1,           0,           0],
                    [           0,  np.cos(rx), -np.sin(rx)],
                    [           0,  np.sin(rx),  np.cos(rx)]])

    Ry = np.array([[  np.cos(ry),           0,  np.sin(ry)],
                    [           0,           1,           0],
                    [ -np.sin(ry),           0,  np.cos(ry)]])

    Rz = np.array([[  np.cos(rz), -np.sin(rz),           0],
                    [  np.sin(rz),  np.cos(rz),           0],
                    [           0,           0,           1]])

    # Applying the rotation
    self.R = np.dot(Rz, np.dot(Ry, Rx))
    self.coords = np.matmul(self.R, self.coords)

    self.rotation = self.R[:,0:2].T.reshape((1, 6))

  def projection2D(self):
    """
    Projects 3D shape onto 2D plane.
    """
    # Removing z axis
    self.coords2D = self.coords[0:, :]
    coords2D_shape = np.shape(self.coords2D)

    # Adding defects to the shape
    if self.defect != 0:
        iters = int(self.defect * coords2D_shape[0])

        for _ in range(iters):
            self.coords2D = np.delete(self.coords2D, np.random.randint(coords2D_shape[0]-1), 0)

    # Translating the shape around randomly
    # self.coords2D[0, :] += np.random.uniform(-self.a, self.a) * 2
    # self.coords2D[1, :] += np.random.uniform(-self.a, self.a) * 2

    # Creating the plot
    # self.fig = plt.figure(figsize=(3, 3))
    self.ax = fig.add_subplot(1, 1, 1)
    self.ax.set_axis_off()

    lim = 10
    self.lims = [-lim, lim]
    self.ax.set_xlim(self.lims)
    self.ax.set_ylim(self.lims)
    self.ax.set_aspect('equal', adjustable='box')
    self.ax.scatter(self.coords2D[0, :], self.coords2D[1, :], s=2, c="white", alpha=self.alpha)

  def save_projection(self, save_path=os.path.join(os.getcwd(), "Simulated Data"), file_name = "default", file_type="png", blur=4, iter=0, aug=False):
    """
    Saves the 2D projection.
    
    Inputs:
      save_path: String, folder in which to save the projections
      file_name: String, desired name for the projection's file name
      file_type: String, what file type to save the file as
      blur: Integer, strength of Gaussian blur
      iter: Integer, number of cubes with the same properties + 1
      aug: Bool, whether or not to save image augmentations
    """
    if not os.path.exists(save_path):
      os.makedirs(save_path)

    if file_name == "default":
      file_name = self.name + (" %s" % (iter))

    # Creating the image from the canvas
    fig.canvas.draw()
    img = PIL.Image.frombytes("RGB", fig.canvas.get_width_height(), fig.canvas.tostring_rgb())
    img = img.filter(ImageFilter.GaussianBlur(radius=blur))

    self.ax.imshow(img, extent=[0, img_width, 0, img_height])
    noise_param = 7000

    spread = 55
    noise = map_params(np.random.poisson(10000, (noise_param, 2)), np.array([-spread, img_width + spread]))
    self.ax.scatter(noise[:,0], noise[:,1], s=1, c="white", alpha=0.035)
    self.ax.set_xlim([0, img_width])
    self.ax.set_ylim([0, img_height])

    fig.canvas.draw()
    img = PIL.Image.frombytes("RGB", fig.canvas.get_width_height(), fig.canvas.tostring_rgb())

    # Where to save
    blur_name = file_name + (" B%s.%s" % (blur, file_type))
    rotation6d = self.rotation.tolist()[0]
    blur_path = os.path.join(save_path, "%s_%s" % (self.name.split()[0], rotation6d))

    if not os.path.exists(blur_path):
      os.makedirs(blur_path)

    blur_file_name = os.path.join(blur_path, blur_name)

    factor = 0.18
    low_x = img_width * factor
    high_x = img_width - low_x
    low_y = img_height * factor
    high_y = img_height - low_y
    img = img.resize((img_width, img_height))
    img = img.crop((low_x, low_y, high_x, high_y))
    img = img.resize((img_width, img_height))
    img.save(blur_file_name)

    fig.clf()

Creating the subclasses for specific shapes which inherits from the shape3D class.

In [9]:
class cube3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a cube.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a cube and include that in the object name.
    """
    self.name = "Cube"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    x0 = self.a * np.arange(-(self.width-1)/2, self.width/2, 1)
    self.coords = np.reshape(np.meshgrid(x0, x0, x0), (3, -1))
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class tetra3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a tetrahedron.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a tetrahedron and include that in the object name.
    """
    if width < 2:
      width = 2
      print("Width must be greater than or equal to 2, setting width to 2.")
    if width > 19:
      width = 19
      print("Width must be less than or equal to 19, setting width to 19.")

    self.name = "Tetrahedron"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Tetrahedron")
    shape_txt = "Tetra%s_Verts.txt" % self.width
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class octa3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for a octahedron.
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a octahedron and include that in the object name.
    """
    if width < 2:
      width = 2
      print("Width must be greater than or equal to 2, setting width to 2.")
    if width > 19:
      width = 19
      print("Width must be less than or equal to 19, setting width to 19.")

    self.name = "Octahedron"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Octahedron")
    shape_txt = "Octa%s_Verts.txt" % self.width
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

class torus3D(shape3D):
  """
  This class inherits from the shape3D class and is specifically for an arbitrarily modified torus (to remove symmetries).
  """
  def __init__(self, width=10, rx=0, ry=0, rz=0, a=1, defect=0, pos_error=0):
    """
    Create a modified torus and include that in the object name.
    """
    self.name = "Torus"
    super().__init__(width, rx, ry, rz, a, defect, pos_error)

  def generate(self):
    """
    Generates all xyz coordinates.
    """
    shape_folder = os.path.join(cwd, "Vertices", "Misc Shapes")
    shape_txt = "Torus_Verts.txt"
    self.coords = np.loadtxt(os.path.join(shape_folder, shape_txt))
    self.coords *= self.a
    self.coords += np.random.normal(0, self.pos_error, np.shape(self.coords)) * self.a

# Generating the images.

The following class is used to generate simulated nanoparticles with the specified parameters. The images are generated, saved, and each subfolder is checked to confirm the images were successfully generated.

In [34]:
class image_generator:
  """
  This class generates images of nanoparticles with the specified parameters for
  its shape and orientation.
  """
  def __init__(self, parent_folder_name, dataset='Train', shape_classes=[cube3D, tetra3D, octa3D], widths=[12], lattice_spacing=0.85, blur=1, iters=1, xyr=[0, 45], dr=15):
    """
    Initialises the class.

    Inputs:
      parent_folder_name: String, folder in which to save all the subfolders of images
      dataset: String, one of ["Train", "Valid", "Test"]
      shape_classes: List, list of shape3D objects to be generated
      widths: List, list of integer widths for nanoparticles
      lattice_spacing: Float, lattice spacing to use for nanoparticles
      blur: Integer, strength of Gaussian blur
      iters: Integer, number of shapes to generate per rotation and shape
      xyr: List, list with two numbers to indicate angle range for rotations
      dr: Float, step between each angle
    """
    self.dataset = dataset
    self.shape_classes = shape_classes
    self.widths = widths
    self.lattice_spacing = lattice_spacing
    self.blur = blur
    self.iters = iters
    self.rxy = generate_params(xyr[0], xyr[1], dr)

    self.parent_folder_name = "%s W %s-%s RX %s-%s RY %s-%s" % (name, int(self.widths[0]), int(self.widths[-1]), int(self.rxy[0]), int(self.rxy[-1]), int(self.rxy[0]), int(self.rxy[-1]))
    self.parent_folder_name = parent_folder_name
    self.folder_name = "%s DR%s" % (self.dataset, dr)
    self.dataset_path = os.path.join(cwd, self.parent_folder_name, self.folder_name)

  def print_num_imgs(self):
    """
    Print the number of images to be generated.
    """
    self.num_rots = len(self.rxy) ** 2
    self.num_imgs = self.num_rots * len(self.shape_classes) * len(self.widths) * self.iters
    print("%s/%s: %s images will be generated belonging to %s orientations (%s each)." % (self.parent_folder_name, self.folder_name, self.num_imgs, self.num_rots, self.num_imgs // self.num_rots))
  
  def run(self, aug=False):
    """
    Generate the images.

    Inputs:
      aug: Boolean, whether or not to save image augmentations
    """
    counter = 1
    for shape_class in self.shape_classes:
      for width in self.widths:
        for angle_y in self.rxy:
          for angle_x in self.rxy:
            for i in range(self.iters):
              print("\r%s/%s: Generating image %s of %s." % (self.parent_folder_name, self.folder_name, counter, self.num_imgs), end='', flush=True)
              shape = shape_class(width, angle_x, angle_y, 0, a=self.lattice_spacing, defect=np.random.uniform(0.03, 0.20), pos_error=np.random.uniform(0.03, 0.045))
              shape.save_projection(save_path=self.dataset_path, file_type=file_type, blur=blur, iter=i, aug=aug)

              counter += 1

    print("\r%s/%s: Generated %s images belonging to %s orientations (%s each).\n" % (self.parent_folder_name, self.folder_name, self.num_imgs, self.num_rots, self.num_imgs // self.num_rots), end='', flush=True)

    if aug:
      print("Images were augmented so 3x the number of images were generated.")
    
  def check_imgs(self):
    """
    Check all images were generated.
    """
    rotations = os.listdir(self.dataset_path)
    imgs = []

    for rotation in rotations:
      imgs.append(len([img for img in os.listdir(os.path.join(self.dataset_path, rotation)) if img.endswith(file_type)]))

    imgs_generated = sum(imgs)
    print("%s/%s: %s images generated, %s expected (Difference of %s)." % (self.parent_folder_name, self.folder_name, self.num_imgs, imgs_generated, (abs(self.num_imgs - imgs_generated))))

Configuring the properties of the lattices to be generated and where to save the images.

In [35]:
# Properties of the shape and its projection
shape_classes = [cube3D, tetra3D, octa3D]
widths = generate_params(12, 13, 1)
lattice_spacing = 0.85 # Pt nanoparticle lattice spacing: 0.24nm, atom size is ~0.1nm | 0.75 for cube
blur = 1 # Strength of Gaussian blur

xyr_train = [0, 45]
xyr_valid = [0, 45]
xyr_test = [0, 45]

dr_train = [2.5]
dr_valid = [2.5]
dr_test = [2.5]

iters = 1
iters_train = 4 * iters
iters_valid = iters
iters_test = iters

name = "Mixed Dataset"
parent_folder_name = "%s W %s-%s RX %s-%s RY %s-%s" % (name, int(widths[0]), int(widths[-1]), int(xyr_train[0]), int(xyr_train[-1]), int(xyr_train[0]), int(xyr_train[-1]))

In [37]:
train_img_generators = []
valid_img_generators = []
test_img_generators = []

for dr in dr_train:
  img_generator = image_generator(parent_folder_name=parent_folder_name, dataset='Train', shape_classes=shape_classes, widths=widths, lattice_spacing=lattice_spacing, blur=blur, iters=iters_train, xyr=xyr_train, dr=dr)
  img_generator.print_num_imgs()
  train_img_generators.append(img_generator)

for dr in dr_valid:
  img_generator = image_generator(parent_folder_name=parent_folder_name, dataset='Valid', shape_classes=shape_classes, widths=widths, lattice_spacing=lattice_spacing, blur=blur, iters=iters_valid, xyr=xyr_valid, dr=dr)
  img_generator.print_num_imgs()
  valid_img_generators.append(img_generator)

for dr in dr_test:
  img_generator = image_generator(parent_folder_name=parent_folder_name, dataset='Test', shape_classes=shape_classes, widths=widths, lattice_spacing=lattice_spacing, blur=blur, iters=iters_test, xyr=xyr_test, dr=dr)
  img_generator.print_num_imgs()
  test_img_generators.append(img_generator)

Mixed Dataset W 12-13 RX 0-45 RY 0-45/Train DR2.5: 8664 images will be generated belonging to 361 orientations (24 each).
Mixed Dataset W 12-13 RX 0-45 RY 0-45/Valid DR2.5: 2166 images will be generated belonging to 361 orientations (6 each).
Mixed Dataset W 12-13 RX 0-45 RY 0-45/Test DR2.5: 2166 images will be generated belonging to 361 orientations (6 each).


Generating the images.

In [15]:
begin_time = datetime.datetime.now()

print("Now generating training dataset images...")
for train_img_generator in train_img_generators:
  train_img_generator.run()

# print("\nNow generating validation dataset images...")
# for valid_img_generator in valid_img_generators:
#   valid_img_generator.run()

# print("\nNow generating testing dataset images...")
# for test_img_generator in test_img_generators:
#   test_img_generator.run()

plt.close()

dt = datetime.datetime.now() - begin_time
dt_m = int(dt.total_seconds() // 60)
dt_s = int(dt.total_seconds() - (dt_m*60))
print("\n\nDone! The time it took is %sm %ss." % (dt_m, dt_s))

Now generating training dataset images...
Mixed Dataset W 12-13 RX 0-45 RY 0-45/Train DR2.5: Generated 8664 images belonging to 361 orientations (24 each).


Done! The time it took is 12m 55s.


Confirming the images were generated and saved successfully.

In [38]:
for train_img_generator in train_img_generators:
  train_img_generator.check_imgs()

for valid_img_generator in valid_img_generators:
  valid_img_generator.check_imgs()

for test_img_generator in test_img_generators:
  test_img_generator.check_imgs()

Mixed Dataset W 12-13 RX 0-45 RY 0-45/Train DR2.5: 8664 images generated, 8664 expected (Difference of 0).
Mixed Dataset W 12-13 RX 0-45 RY 0-45/Valid DR2.5: 2166 images generated, 2166 expected (Difference of 0).
Mixed Dataset W 12-13 RX 0-45 RY 0-45/Test DR2.5: 2166 images generated, 2166 expected (Difference of 0).


The specified parameters are saved into a text file.

In [14]:
f = open(os.path.join(cwd, parent_folder_name, "Configuration.txt"), "w")
f.write("RX [%s, %s]\n" % (xyr_train[0], xyr_train[-1]))
f.write("RY [%s, %s]\n" % (xyr_train[0], xyr_train[-1]))
f.write("Widths %s\n" % widths)
f.write("Lattice spacing %s\n" % lattice_spacing)
f.write("blur %s\n" % blur)
f.write("iters %s" % iters)
f.close()